In [1]:
from IPython.display import display
import pandas as pd

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [2]:
import json

with open("data/import_export_ports_francs.json", "r") as f:
    data = json.load(f)

In [3]:
SORT_LABELS = ["colonies", "Monde", "France", "Fraude ?", "Re-exp ou fraude ?"]
PALETTE = ["purple", "darkblue", "olive", "lightgrey", "darkgrey"]

In [13]:
VegaLite({
  "title": "Répartition des imports/exports de Dunkerque en 1789 par produits (SITC simpl)",
  "data": {"values": data["Dunkerque"]},
  "mark": {"type":"bar", "tooltip": {"content": "data"}},
  "encoding": {
    "x": {"field": "value", "type": "quantitative", "title": "Valeur (Lt)"},
    "column": {"field": "group"},
    "y": {"field": "product", "sort": "-x", "title": "Produits"},
    "color": {
        "field": "partner", 
        "scale": {
            "domain": SORT_LABELS,
            "range": PALETTE
        },
        "sort": SORT_LABELS,
        "type": "nominal"
    }
  }
})

In [5]:
VegaLite({
  "data": {"values": data["Dunkerque"]},
  "transform": [{
      "aggregate": [{
       "op": "sum",
       "field": "value",
       "as": "total_product"
      }],
      "groupby": ["partner"]
  }],
  "mark": {"type":"bar", "tooltip": {"content": "data"}},
  "encoding": {
    "x": {"field": "total_product", "type": "quantitative", "title": "Valeur (Lt)"},
    "y": {"field": "partner", "sort": SORT_LABELS, "title": "Partner"},
    "color": {
        "field": "partner", 
        "sort": SORT_LABELS, 
        "type": "nominal", 
        "scale": {
            "domain": SORT_LABELS,
            "range": PALETTE
        }
    }
  }
})

In [71]:
from itertools import groupby

def pyramide_port_franc(port_franc, data):
    report = data[port_franc]
    key = lambda d: (d['group'], d['partner'])
    sd = sorted(report, key=key)
    max_scale = max([sum(t["value"] for t in totals) for group, totals in groupby(sd, key=key)])
    max_total = max([sum(t["value"] for t in totals) for exp_imp, totals in groupby(sd, key=lambda d: d['group'])])
    VegaLite({
      "title": "{}\n Répartition des imports/exports en 1789 par produits (SITC simpl)".format(port_franc.upper()),
      "data": {"values": report},
      "spacing": 0,
      "vconcat": [{"hconcat": [{
            "transform": [{
              "filter": {"field": "group", "equal": "import"}
            }],
            "title": "import",
            "mark": {"type":"bar", "tooltip": {"content": "data"}},
            "encoding": {
              "y": {
                "field": "product", "title": None, "sort": "-x", "type": "ordinal"
              },
              "x": {
                "field": "value",
                "type": "quantitative",
                "title": "Valeur (Lt)",
                  "sort": "descending"
              },
              "color": {
                "field": "partner",
                "type": "nominal", 
                "sort": SORT_LABELS, 
                "scale": {
                    "domain": SORT_LABELS,
                    "range": PALETTE
                },

              }
            }
          },
          {
            "transform": [{
              "filter": {"field": "group", "equal": "export"}
            }],
            "title": "export",
            "mark": {"type":"bar", "tooltip": {"content": "data"}},
            "encoding": {
              "y": {
                "field": "product", "sort": "-x", "title": None, "axis": None, "type": "ordinal"

              },
              "x": {
                "field": "value",
                "type": "quantitative",
                "title": "Valeur (Lt)",
                "sort": "ascending"
              },
              "color": {
                "field": "partner",
                "type": "nominal", 
                "sort": SORT_LABELS, 
                "scale": {
                    "domain": SORT_LABELS,
                    "range": PALETTE
                },
              }
            }
          }
          ]}, {"hconcat": [{
            "transform": [
                {
              "filter": {"field": "group", "equal": "import"}
            },
            {
                  "aggregate": [{
                   "op": "sum",
                   "field": "value",
                   "as": "total_value"
                  }],
                  "groupby": ["partner"]
            }
            ],
            "mark": {"type":"bar", "tooltip": {"content": "data"}},
            "encoding": {
              "y": {
                "field": "partner", 
                "title": None, 
                "type": "ordinal",
                "sort": SORT_LABELS,
                "scale": {"domain": SORT_LABELS},

              },
              "x": {
                "field": "total_value",
                "type": "quantitative",
                "title": "Valeur tous produits (Lt)",
                "sort": "descending",
                "scale": {"domain": [0, max_scale]}
              },
              "color": {
                "field": "partner",
                "type": "nominal", 
                "sort": SORT_LABELS, 
                "scale": {
                    "domain": SORT_LABELS,
                    "range": PALETTE
                },
              }
            }
          },
          {
            "transform": [
                {
              "filter": {"field": "group", "equal": "export"}
            },
            {
                  "aggregate": [{
                   "op": "sum",
                   "field": "value",
                   "as": "total_value"
                  }],
                  "groupby": ["partner"]
            }
            ],
            "mark": {"type":"bar", "tooltip": {"content": "data"}},
            "encoding": {
              "y": {
                "field": "partner", 
                "title": None, 
                "axis": None, 
                "type": "ordinal",
                "sort": SORT_LABELS,
                "scale": {"domain": SORT_LABELS},

              },
              "x": {
                "field": "total_value",
                "type": "quantitative",
                "title": "Valeur tous produits (Lt)",
                "sort": "ascending", 
                "scale": {"domain": [0, max_scale]}
              },
              "color": {
                "field": "partner",
                "type": "nominal", 
                "sort": SORT_LABELS, 
                "scale": {
                    "domain": SORT_LABELS,
                    "range": PALETTE
                },
              }
            }
          }
          ]},
          {"hconcat": [{
            "transform": [
            {
              "filter": {"field": "group", "equal": "import"}
            },
                        {
              "filter": "indexof(datum.partner,'?')==-1"
            },

            {
                  "aggregate": [{
                   "op": "sum",
                   "field": "value",
                   "as": "total_value"
                  }]
            }
            ],
            "mark": {"type":"bar", "tooltip": {"content": "data"}},
            "encoding": {
              "x": {
                "field": "total_value",
                "type": "quantitative",
                "title": "Valeur tous produits (Lt)",
                "sort": "descending",
                 "scale": {"domain": [0, max_total]}
              },
            }
          },
          {
            "transform": [
            {
              "filter": {"field": "group", "equal": "export"}
            },
                {
              "filter": "indexof(datum.partner,'?')==-1"
            },
            {
                  "aggregate": [{
                   "op": "sum",
                   "field": "value",
                   "as": "total_value"
                  }]
            }
            ],
            "mark": {"type":"bar", "tooltip": {"content": "data"}},
            "encoding": {
              "x": {
                "field": "total_value",
                "type": "quantitative",
                "title": "Totaux (Lt)",
                "sort": "ascending",
                "scale": {"domain": [0, max_total]}
              }
            }
          }
          ]},
          
        ]
    }
    )

In [72]:
for city in ["Dunkerque", "Bayonne", "Marseille", "Lorient"]:
    pyramide_port_franc(city, data)

# Continuations

ajouter la valeur totale pour tout le graphique pour chacun des 5 (6 avec ceux dont on est sûrs que c'est de la fraude) partners
entrepôt ou fraude (résidu) - pour la première ligne c'est sûr que c'est de la fraude

regarder navigo :peut-on distinguer entre entrepôt légitime et fraude ? Y a-t-il suffisamment de navires (quel tonnage ?) qui partent avec ce genre de produits ?  --> voir qui travaille là-dessus

pour les autres ports francs on a les exports, donc pas besoin de calculer un résidu  --> plus facile de distinguer la fraude de l'entrepôt

regarder quand commence cet entrepôt / fraude --> lire la thèse de Christian p. 167 - 169, d'après Christian on pourrait mettre en évidence que ça commence vers 1730

Plusieurs séries temporelles ?